In [1]:
!pip install pdfplumber PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive, files
import os

import pdfplumber
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import pdfplumber
import fitz  # PyMuPDF
import re

def extract_text_with_pymupdf(pdf_path):
    """Extrai texto usando PyMuPDF (fitz) como alternativa."""
    text = []
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text.append(page.get_text("text"))
        return "\n".join(text)
    except Exception as e:
        print(f"Erro com PyMuPDF: {pdf_path} -> {e}")
        return None

def pdfs_to_txt_in_directory(input_dir, output_dir, skip_until):
    """
    Converte PDFs em .txt, recriando a estrutura de pastas, mas pulando
    diretórios que começam com números de 1 até skip_until.
    Além disso, processa apenas arquivos dentro da pasta "2023".

    Args:
        input_dir (str): Diretório contendo os PDFs.
        output_dir (str): Diretório onde os arquivos .txt serão salvos.
        skip_until (int): Número até o qual as pastas serão ignoradas.
    """
    for root, subdirs, files in os.walk(input_dir, topdown=True):
        relative_path = os.path.relpath(root, input_dir)
        folder_name = os.path.basename(root)

        # Pula pastas que começam com número de 1 até skip_until seguido de espaço
        match = re.match(r'^(\d+)\s', folder_name)
        if match:
            folder_number = int(match.group(1))
            if 1 <= folder_number <= skip_until:
                print(f"Pulado diretório: {root}")
                subdirs[:] = []  # Evita entrar nas subpastas desse diretório
                continue

        # Se o diretório "2023" não estiver no caminho, pula
        if "2023" not in relative_path.split(os.sep):
            continue

        converted_files = []

        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)

                txt_filename = os.path.splitext(file)[0] + '.txt'
                txt_path = os.path.join(output_subdir, txt_filename)

                if os.path.exists(txt_path):
                    continue  # Pula se o arquivo já existe

                text_extracted = None

                # Tenta extrair com pdfplumber
                try:
                    with pdfplumber.open(pdf_path) as pdf:
                        text_extracted = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
                except Exception as e:
                    print(f"Erro com pdfplumber: {pdf_path} -> {e}")

                # Se pdfplumber falhar, tenta com PyMuPDF
                if not text_extracted:
                    text_extracted = extract_text_with_pymupdf(pdf_path)

                if text_extracted:
                    with open(txt_path, 'w', encoding='utf-8') as txt_file:
                        txt_file.write(text_extracted)
                    converted_files.append(txt_path)
                else:
                    print(f"Erro: {pdf_path} não pôde ser convertido.")

        # Após processar a pasta, imprime os arquivos convertidos
        if converted_files:
            print(f"Convertidos na pasta {root}:")
            for txt_file in converted_files:
                print(f" - {txt_file}")

# Exemplo de uso
input_directory = "/content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/"
output_directory = "/content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos txt/"
skip_until = 46 # Número até o qual as pastas serão puladas

pdfs_to_txt_in_directory(input_directory, output_directory, skip_until)

Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/2 DESCONTO SIMPLIFICADO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/3 PRAZO PARA APRESENTAÇÃO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/4 MULTA POR ATRASO NA ENTREGA DA DECLARAÇÃO OU NÃO APRESENTAÇÃO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/5 PREENCHIMENTO IRPF 2024
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/6 TRANSMISSAO DA DECLARAÇÃO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/7 LOCAL DE APRESENTAÇÃO
Pulado diretório: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/8 DECLARAÇÃO DE EXERCÍCIOS ANTERIORES
Pulado diretório: /content/drive/Sharedd

Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/47 ALIENAÇÃO DE BENS E DIREITOS/2023/15463720012202117_6739752.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/47 ALIENAÇÃO DE BENS E DIREITOS/2023/18239001600200911_6940710.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/47 ALIENAÇÃO DE BENS E DIREITOS/2023/10855002201200567_6888321.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/47 ALIENAÇÃO DE BENS E DIREITOS/2023/11040721535201803_6789402.pdf -> No /Root object! - Is this really a PDF?


...

Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/11762720038201753_6954302.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/10507000731200873_6831036.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/10983902687201461_6833604.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/10680720570201394_6889528.pdf -> No /Root object! - Is this really a PDF?


Erro com pdfplumber: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/10680720568201315_6899329.pdf -> No /Root object! - Is this really a PDF?
Convertidos na pasta /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023:
 - /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos txt/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/15889000185200928_6764410.txt
 - /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos txt/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/10940720703201233_6737463.txt
 - /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos txt/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/11516722368201361_6815109.txt
 - /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos txt/49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS/2023/19515000018200803_6812399.txt
 - /content/drive/Shareddrives/RAG Rece